The objective of this task is to predict keypoint positions on face images. This can be used as a building block in several applications, such as:

tracking faces in images and video
analysing facial expressions
detecting dysmorphic facial signs for medical diagnosis
biometrics / face recognition
Detecing facial keypoints is a very challenging problem.  Facial features vary greatly from one individual to another, and even for a single individual, there is a large amount of variation due to 3D pose, size, position, viewing angle, and illumination conditions. Computer vision research has come a long way in addressing these difficulties, but there remain many opportunities for improvement.

In [0]:
#Mounting the Google Drive for accessing the dataset
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
#Import the packages
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from skimage import data, io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Convolution2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, Conv2D,MaxPool2D, ZeroPadding2D

In [0]:
#Getting the dataset and storing into one dataframe
df_training = pd.read_csv("/content/gdrive/My Drive/facial-keypoints-detection/training.csv")

In [0]:
#Filling the null or NA data in Dataframe
df_training.fillna(method="ffill", inplace=True)

In [0]:
#Dividing the data into Source and target variables
X = df_training.Image
y = df_training.drop('Image', axis = 1)
X.shape, y.shape

In [0]:
#Write a function to Normalize the data of better training
def convertImage(Imagedata):
  newData = [np.array(i.split(' ')).astype(np.float32) for i in Imagedata]
  newData = np.array(newData).reshape(Imagedata.shape[0], 96, 96, 1)
  newData /= 255
  return newData

In [0]:
#Normalizing the data
X = convertImage(X)
X.shape

In [0]:
#Visualizing the image with Image features
plt.imshow(X[0].reshape(96,96),cmap='gray')
t = y.iloc[0]
plt.scatter(t[0::2], t[1::2], c='red', marker='x')
plt.show()

In [0]:
#Function to define a model
def create_model():
  model = Sequential()
  model.add(Convolution2D(32, (3,3), padding='same', use_bias=False, input_shape=(96,96,1)))

  model.add(LeakyReLU(alpha = 0.1))
  model.add(BatchNormalization())

  model.add(Convolution2D(32, (3,3), padding='same', use_bias=False))
  model.add(LeakyReLU(alpha = 0.1))
  model.add(BatchNormalization())
  model.add(MaxPool2D(pool_size=(2, 2)))

  model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
  model.add(LeakyReLU(alpha = 0.1))
  model.add(BatchNormalization())

  model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
  model.add(LeakyReLU(alpha = 0.1))
  model.add(BatchNormalization())
  model.add(MaxPool2D(pool_size=(2, 2)))

  model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
  model.add(LeakyReLU(alpha = 0.1))
  model.add(BatchNormalization())

  model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
  model.add(LeakyReLU(alpha = 0.1))
  model.add(BatchNormalization())
  model.add(MaxPool2D(pool_size=(2, 2)))

  model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
  # model.add(BatchNormalization())
  model.add(LeakyReLU(alpha = 0.1))
  model.add(BatchNormalization())

  model.add(Convolution2D(128, (3,3),padding='same', use_bias=False))
  model.add(LeakyReLU(alpha = 0.1))
  model.add(BatchNormalization())
  model.add(MaxPool2D(pool_size=(2, 2)))

  model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
  model.add(LeakyReLU(alpha = 0.1))
  model.add(BatchNormalization())

  model.add(Convolution2D(256, (3,3),padding='same',use_bias=False))
  model.add(LeakyReLU(alpha = 0.1))
  model.add(BatchNormalization())
  model.add(MaxPool2D(pool_size=(2, 2)))

  model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
  model.add(LeakyReLU(alpha = 0.1))
  model.add(BatchNormalization())

  model.add(Convolution2D(512, (3,3), padding='same', use_bias=False))
  model.add(LeakyReLU(alpha = 0.1))
  model.add(BatchNormalization())


  model.add(Flatten())
  model.add(Dense(512,activation='relu'))
  model.add(Dropout(0.1))
  model.add(Dense(30))
  return model

In [0]:
#Accessing TPU on colab
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [0]:
#Create model with TPU
with tpu_strategy.scope(): # creating the model in the TPUStrategy scope means we will train the model on the TPU
  model = create_model()
model.summary()

In [0]:
#Compiling the model with Adam optimizer, loss with MSE and accuracy for matrics
model.compile(loss='mean_squared_error',optimizer='Adam',metrics=['accuracy'])

In [0]:
#Fitting the model
model.fit(X, y, batch_size=256, epochs=100)

In [0]:
#Accessing the data for testing
df_testing = pd.read_csv("/content/gdrive/My Drive/facial-keypoints-detection/test.csv")
df_testing.shape

In [0]:
#Normalize the testing data
test_image = convertImage(df_testing.Image)
test_image.shape

In [0]:
#Predicting based on test data
test_predict = model.predict(test_image)

In [0]:
test_predict[0]

In [0]:
plt.imshow(test_image[0].reshape(96,96),cmap='gray')
t = test_predict[0]
plt.scatter(t[0: :2], t[1: :2], c='red', marker='x')
plt.show()

In [0]:
plt.imshow(X[0].reshape(96,96),cmap='gray')
t = y.iloc[0]
plt.scatter(t[0::2], t[1::2], c='red', marker='x')
plt.show()